### 1. 행정구역별 15세 미만 인구수
### 2. 행정구역별 65세 이상 인구수

In [17]:
# 읍면동 별 인구수 데이터를 불러온다.
import pandas as pd
df = pd.read_excel('../data/raw/202506_읍면동_연령별인구수.xlsx')

In [18]:
# 행정구역에 NaN값 처리
# 본래 엑셀에서는 15세 미만과 65세 이상에 대해, 하나의 행에만 행정구역 이름이 들어가 있어, 정확히 절반은 NaN으로 떠버림
# 따라서, NaN 바로 앞에 있는 행정구역 데이터로 fillna를 수행

df['행정구역별(읍면동)'] = df['행정구역별(읍면동)'].fillna(method='ffill')
display(df.head())

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_1644\1054061980.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['행정구역별(읍면동)'] = df['행정구역별(읍면동)'].fillna(method='ffill')


,행정구역별(읍면동),연령별,총인구(명),내국인(명)
0,사직동,15세미만,619,611
1,사직동,65세이상,1826,1796
2,삼청동,15세미만,130,123
3,삼청동,65세이상,577,571
4,부암동,15세미만,683,681


In [19]:
# 컬럼 이름 수정
df.rename(columns={'행정구역별(읍면동)':'행정구역', '연령별': '연령대'}, inplace=True)
df.head()

,행정구역,연령대,총인구(명),내국인(명)
0,사직동,15세미만,619,611
1,사직동,65세이상,1826,1796
2,삼청동,15세미만,130,123
3,삼청동,65세이상,577,571
4,부암동,15세미만,683,681


In [20]:
# 총인구과 내국인 열을 String값에서 int값으로 수정한다. 만약 NaN값이 있다면 0으로 채운다.
df['총인구(명)'] = pd.to_numeric(df['총인구(명)'], errors='coerce').fillna(0)
df['내국인(명)'] = pd.to_numeric(df['내국인(명)'], errors='coerce').fillna(0)

# 1. 행정구역별 65세 이상 인구수
pop_65_plus = df[df['연령대'] == '65세이상'].groupby('행정구역')['총인구(명)'].sum()
display(pop_65_plus.head())

# 2. 행정구역별 15세 미만 인구수
pop_under_15 = df[df['연령대'] == '15세미만'].groupby('행정구역')['총인구(명)'].sum()
display(pop_under_15.head())

행정구역
　　　　　　가경동    7305.0
　　　　　　가곡동    2364.0
　　　　　　가곡면    1184.0
　　　　　　가남읍    3943.0
　　　　　　가능동    5152.0
Name: 총인구(명), dtype: float64

행정구역
　　　　　　가경동    7100.0
　　　　　　가곡동     475.0
　　　　　　가곡면     113.0
　　　　　　가남읍    1099.0
　　　　　　가능동    1867.0
Name: 총인구(명), dtype: float64

### 3. 행정구역별 외국인 인구수 (미숙한 한국어는 재난 발생시에 위험을 높힐 수 있기에 점수 계산에 포함)

In [21]:
# 행정구역의 총 인구수와 내국인 수가 적힌 데이터를 불러온다.
df2 = pd.read_excel('../data/raw/202506_읍면동_총인구수.xlsx')
df2.head()

,행정구역별(읍면동),연령별,총인구(명),내국인(명)
0,사직동,합계,8411,8138
1,삼청동,합계,2060,1994
2,부암동,합계,9057,8712
3,평창동,합계,16392,16098
4,무악동,합계,7446,7398


In [22]:
# 마찬가지로 String 값을 int값으로 수정한다.
df2['총인구(명)'] = pd.to_numeric(df2['총인구(명)'], errors='coerce').fillna(0)
df2['내국인(명)'] = pd.to_numeric(df2['내국인(명)'], errors='coerce').fillna(0)

# 총인구수 - 내국인 인구수 = 외국인 인구수
df2['외국인(명)'] = df2['총인구(명)'] - df2['내국인(명)']

# 이제 행정구역별 외국인 수와 총인구수(참고용)을 구한다.
foreigners_by_district_df2 = df2.groupby('행정구역별(읍면동)')[['외국인(명)', '총인구(명)']].sum()

display(foreigners_by_district_df2.head())

,외국인(명),총인구(명)
행정구역별(읍면동),,
가경동,684.0,52880.0
가곡동,113.0,7247.0
가곡면,38.0,2492.0
가남읍,1592.0,15434.0
가능동,530.0,23424.0


### 4. 행정구역별 장애인 인구수

In [46]:
#

### 5. 데이터 export

In [23]:
# 오리지날 데이터를 각각의 변수에 copy
pop65 = pop_65_plus.copy()
pop15 = pop_under_15.copy()
popFo = foreigners_by_district_df2.copy()

# Merge를 수행하기 전, 컬럼 명들을 통일시켜 준다.
pop65_df = pop65.reset_index().rename(columns={'행정구역': '행정구역', '총인구(명)': '65세이상인구수'})
pop15_df = pop15.reset_index().rename(columns={'행정구역': '행정구역', '총인구(명)': '15세미만인구수'})
popFo_df = popFo.reset_index().rename(columns={'행정구역별(읍면동)': '행정구역', '외국인(명)': '외국인수', '총인구(명)':'총인구'})

# Merge 수행
total = pd.merge(pop65_df, pop15_df, on='행정구역', how='outer')
total = pd.merge(total, popFo_df, on='행정구역', how='outer')

display(total.head())

,행정구역,65세이상인구수,15세미만인구수,외국인수,총인구
0,가경동,7305.0,7100.0,684.0,52880.0
1,가곡동,2364.0,475.0,113.0,7247.0
2,가곡면,1184.0,113.0,38.0,2492.0
3,가남읍,3943.0,1099.0,1592.0,15434.0
4,가능동,5152.0,1867.0,530.0,23424.0


In [25]:
from pathlib import Path

# 저장할 폴더와 파일명 지정
processed_dir = Path('..') / 'data' / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)   # 폴더가 없으면 생성

output_file = processed_dir / '202506_읍면동_사회취약계층표.xlsx'

# DataFrame을 엑셀로 내보내기
total.to_excel(output_file, index=False)  
print(f"Saved Excel to {output_file.resolve()}")

Saved Excel to C:\HIuniv_Project-hyeonseo\data\processed\202506_읍면동_사회취약계층표.xlsx


## 사회취약계층 계산